<a href="https://colab.research.google.com/github/Nikita0190/AIML-Project-Series/blob/main/PROJECT_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import random
from typing import Dict, List, Tuple
from collections import defaultdict


class USER:
    def __init__(self, name: str, patterns: List[str], responses: List[str]):
        self.name = name
        self.patterns = patterns
        self.responses = responses


class CollegeAdmissionChatbot:
    def __init__(self):
        self.intents: List[USER] = [
            USER("admission_procedure",
                 ["how to apply", "admission process", "application steps"],
                 [
                     "The general admission procedure typically involves:\n1. Researching colleges and programs\n2. Taking standardized tests (SAT/ACT)\n3. Submitting an online application\n4. Paying the application fee\n5. Sending official transcripts\n6. Writing a personal essay\n7. Obtaining letters of recommendation\n8. Completing an interview (if required)\n9. Waiting for admission decision\n10. Accepting an offer and paying a deposit"]),
            # ... (other intents remain the same)
        ]
        self.context: Dict[str, str] = {}
        self.fallback_responses = [
            "I'm not sure I understand. Could you rephrase that?",
            "I don't have specific information on that topic. Can you try asking about admissions, requirements, deadlines, financial aid, majors, campus life, or transfers?",
            "I'm having trouble understanding your question. Can you try asking in a different way?",
            "That's a bit outside my area of expertise. Can you ask something more specifically related to college admissions or campus life?"
        ]
        self.keyword_to_intent = self._build_keyword_index()
        self.conversation_history: List[str] = []
        self.user_details: Dict[str, str] = {}
        self.admission_details: Dict[str, str] = {}
        self.previous_topics: List[str] = []

    def _build_keyword_index(self) -> Dict[str, List[USER]]:
        keyword_to_intent = defaultdict(list)
        for intent in self.intents:
            for pattern in intent.patterns:
                keywords = pattern.split()
                for keyword in keywords:
                    keyword_to_intent[keyword].append(intent)
        return keyword_to_intent

    def preprocess(self, text: str) -> str:
        return text.lower()

    def match_intent(self, text: str) -> Tuple[USER, float]:
        words = text.split()
        intent_counts = defaultdict(int)
        for word in words:
            if word in self.keyword_to_intent:
                for intent in self.keyword_to_intent[word]:
                    intent_counts[intent] += 1
        if not intent_counts:
            return None, 0.0
        matched_intent = max(intent_counts, key=intent_counts.get)
        confidence = intent_counts[matched_intent] / len(words)
        return matched_intent, confidence

    def update_context(self, intent: USER):
        self.context[intent.name] = True

    def generate_response(self, intent: USER, text: str) -> str:
        return random.choice(intent.responses)

    def collect_user_details(self):
        details_to_collect = [
            ("name", "What's your name?"),
            ("age", "What's your age?"),
            ("city", "Which city are you from?"),
            ("state", "Which state do you live in?"),
            ("email", "What's your email address?"),
            ("phone", "What's your phone number?"),
            ("intended_major", "What major are you interested in?"),
            ("target_universities", "Which universities are you considering?")
        ]

        for key, question in details_to_collect:
            while True:
                try:
                    value = input(f"Chatbot: {question}\nYou: ").strip()
                    if not value:
                        raise ValueError("This field cannot be empty.")

                    if key == "age":
                        age = int(value)
                        if age < 14 or age > 100:
                            raise ValueError("Age must be between 14 and 100.")
                    elif key == "email":
                        if not re.match(r"[^@]+@[^@]+\.[^@]+", value):
                            raise ValueError("Invalid email format.")
                    elif key == "phone":
                        if not re.match(r"^\d{10}$", value):
                            raise ValueError("Phone number must be 10 digits.")

                    self.user_details[key] = value
                    break
                except ValueError as e:
                    print(f"Error: {str(e)} Please try again.")

        print("Chatbot: Thank you for providing your details. How can I assist you with college admissions?")

    def get_response(self, text: str) -> str:
        try:
            self.conversation_history.append(f"User: {text}")
            preprocessed_text = self.preprocess(text)
            matched_intent, confidence = self.match_intent(preprocessed_text)

            if matched_intent and confidence > 0.2:
                self.update_context(matched_intent)
                response = self.generate_response(matched_intent, preprocessed_text)
                self.admission_details[matched_intent.name] = response
                self.previous_topics.append(matched_intent.name)
            else:
                response = random.choice(self.fallback_responses)

            self.conversation_history.append(f"Chatbot: {response}")
            return response
        except Exception as e:
            return f"I apologize, but an error occurred: {str(e)}. Could you please try rephrasing your question?"

    def handle_follow_up(self, text: str) -> str:
        try:
            if any(keyword in text.lower() for keyword in ["more", "additional", "clarify", "explain"]):
                last_intent = self.get_last_intent()
                if last_intent:
                    return f"Sure, here's some additional information about {last_intent.name}:\n" + random.choice(
                        last_intent.responses)

            return self.get_response(text)
        except Exception as e:
            return f"I'm sorry, but I encountered an error while processing your request: {str(e)}. Could you please try again?"

    def get_last_intent(self) -> USER:
        for item in reversed(self.conversation_history):
            if item.startswith("Chatbot:"):
                for intent in self.intents:
                    if any(response in item for response in intent.responses):
                        return intent
        return None

    def summarize_conversation(self) -> str:
        summary = "Here's a summary of our conversation:\n\n"

        if self.user_details:
            summary += "Personal Details:\n"
            for key, value in self.user_details.items():
                summary += f"- {key.capitalize()}: {value}\n"
            summary += "\n"

        if self.admission_details:
            summary += "College Admission Information Discussed:\n"
            for topic in self.previous_topics:
                if topic in self.admission_details:
                    summary += f"- {topic.capitalize()}: {self.admission_details[topic][:100]}...\n"

        return summary

    def get_personalized_recommendation(self) -> str:
        recommendation = "Based on your profile and our conversation, here are some personalized recommendations:\n\n"

        if 'intended_major' in self.user_details:
            recommendation += f"1. For your intended major in {self.user_details['intended_major']}, "
            recommendation += "consider researching the following related majors as well: [List related majors]\n"

        if 'target_universities' in self.user_details:
            recommendation += f"2. In addition to {self.user_details['target_universities']}, "
            recommendation += "you might want to look into these universities known for strong programs in your field: [List universities]\n"

        recommendation += "3. Given your background, consider the following scholarship opportunities: [List scholarships]\n"

        return recommendation


def chat_session():
    chatbot = CollegeAdmissionChatbot()
    print("Welcome to the College Admission Chatbot. Let's start by collecting some information about you.")

    chatbot.collect_user_details()

    print(
        "Great! Now, how can I help you with college admissions? You can ask about admission procedures, requirements, deadlines, financial aid, majors, campus life, or transfers.")

    while True:
        try:
            user_input = input("You: ")
            if user_input.lower() in ['exit', 'quit', 'bye']:
                print("\nChatbot: Before you go, here's a summary of our conversation:")
                print(chatbot.summarize_conversation())
                print("\nHere are some personalized recommendations:")
                print(chatbot.get_personalized_recommendation())
                print("Thank you for using the College Admission Chatbot. Goodbye!")
                break

            if user_input.lower() == "summary":
                print("\nChatbot: Here's a summary of our conversation so far:")
                print(chatbot.summarize_conversation())
                continue

            response = chatbot.handle_follow_up(user_input)
            print("Chatbot:", response)

            follow_up = input("Do you need more information on this topic? (yes/no): ")
            if follow_up.lower() == 'yes':
                additional_info = chatbot.handle_follow_up("Tell me more")
                print("Chatbot:", additional_info)
        except KeyboardInterrupt:
            print("\nChatbot: You've interrupted the chat. If you want to exit, please type 'exit', 'quit', or 'bye'.")
        except Exception as e:
            print(f"An unexpected error occurred: {str(e)}. Please try again.")

if __name__ == "__main__":
    chat_session()

Welcome to the College Admission Chatbot. Let's start by collecting some information about you.
Chatbot: What's your name?
You: nikita
Chatbot: What's your age?
You: 20
